#### Setup Step 1 - configuring the RSpace Client. `rspace_client` is available from pip.

It's good practice to store API keys as environment variables rather than hard-coding it.

In [ ]:
!pip install rspace-client==2.0.1
print("Kernel running OK")

In [ ]:
## get your API key and set your RSpace URL. Change this code as needed for your own environment
from kaggle_secrets import UserSecretsClient
api_key_label = "mp_demos_key"
API_KEY = UserSecretsClient().get_secret(api_key_label)
print (f"Retrieved API key {API_KEY[0:4]}...")
URL="https://demos.researchspace.com"

In [ ]:
from rspace_client.eln import eln
import os

api = eln.ELNClient(URL, API_KEY)

## sanity check that that the client is configured correctly
print(api.get_status())

#### Setup Step 2 - adding some test data.

Here we'll add a CSV file to RSpace, containing some synthetic weather-related data.

In [ ]:
import os
data_input_dir='/kaggle/input/rspacedemofiles'
temp_data_path=os.path.join(data_input_dir, 'temp_data.csv')

with open (temp_data_path) as f:
    raw_data_file = api.upload_file(f)['id']
raw_data_file_id= raw_data_file
print(f"Temperature data uploaded to RSpace with ID {raw_data_file_id}")

#### Analysis Step 1 - retrieving dataset

OK, now we can start working with this dataset. If this dataset had been uploaded by a colleague, we could have been notified by Slack, Teams, email or within RSpace itself that this file was available for analysis.

In [ ]:
file_name = "downloaded_"+(api.get_file_info(raw_data_file_id)['name'])
print(file_name)

## retrieve from RSpace - here we are downloading the file
raw_temp_data = api.download_file(raw_data_file_id, file_name)

#### Analysis Step 2 - the analysis

Here is where you do your actual analytis of the data... here we'll just plot the data and generate a summary, saving both to file.

In [ ]:
import pandas as pd;
df = pd.read_csv(file_name)
summary_stats = df.describe()

df = df.set_index('city_id')
plot = df.plot(ylabel='Celsius', title=f'Temperature plots from dataset {raw_data_file_id}')
img_f= f'Temperature_per_city-{raw_data_file_id}'
plot.get_figure().savefig(img_f)

summary_stats_csv = f'{file_name[:file_name.rindex(".")]}-summarystats.csv'
summary_stats.to_csv(summary_stats_csv)

#### Analysis Step 3 - uploading back to RSpace

You can add captions to the file to help describe your analysis

In [ ]:
with open(summary_stats_csv, 'rb') as f:
    summary_file = api.upload_file(f, caption=f"Summary data for {raw_data_file_id}")
    print(f"uploaded id = {summary_file['id']}")
with open(img_f+".png", 'rb') as f:
    uploaded_image = api.upload_file(f, caption=f"City vs temperature for  {raw_data_file_id}")
    print(f"uploaded id = {uploaded_image['id']}") 

There are several  options now:

* You can create an RSpace document, and insert these files, and share the document with your group or colleage. 
* Your colleagues may have already created and shared document describing an experiment that generated these files, in which case you would already have access to a document.

Here we'll go with a simple flow where we create a new RSpace document to share with the rest of our research group.

The content we'll insert will be HTML. However you don't need to figure out how to display the linked files. Just include file links as `<fileId=XXXX>` syntax and RSpace will turn these into formatted links


In [ ]:
new_doc = api.create_document(name=f"Analysis of dataset {raw_data_file_id}")
content = f"""
<p>Analysis of temperature dataset from our standard locations.
<p>No variation between locations:
Raw data: <fileId={raw_data_file_id}>
<p>
Statistical summary: <fileId={summary_file['id']}>
<p>
Location vs temperature: <fileId={uploaded_image['id']}>
"""

updated_doc = api.append_content(new_doc['id'], content)

## a simple utility function so you can get a link to view the updated contents in a browser.
def api_to_browser(link):
    return '/globalId/SD'.join(link.split('/api/v1/documents/'))

print(f"You can view this in a browser at {api_to_browser(updated_doc['_links'][0]['link'])}")


If you're in a  group, you can now share this with your group. You can get your groups' IDs: 

In [53]:
groups = api.get_groups()
for gp in groups:
    print(f"{gp['name']:30}{gp['id']}")
chosen_group = None
#chosen_group = input("please enter a group ID to share with")
chosen_group = chosen_group or groups[0]['id'] ## if not running interactively, choose 1st group

In [ ]:
api.shareDocuments([new_doc['id']], chosen_group, permission="EDIT")

In [ ]:
### tidy up - remove output files
outfile_dir="/kaggle/working"
for root,dirs,files in os.walk(outfile_dir):
    for f in files:
        os.remove(f)
print ("output files removed")